# **Crawling Data**

Sel ini adalah judul utama notebook, yaitu "Crawling Data".

In [ ]:
#@title Twitter Auth Token

#twitter_auth_token = '#ubah dengan auth token' # change this auth token'
#twitter_auth_token = '' # change this auth token
twitter_auth_token = '' # change this auth token

Sel ini berfungsi untuk mengatur token autentikasi Twitter. Anda perlu mengganti string kosong dengan token autentikasi Twitter Anda.

In [ ]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

Sel ini melakukan instalasi pustaka yang diperlukan. Pertama, menginstal `pandas` menggunakan pip. Kemudian, menginstal Node.js yang dibutuhkan oleh `tweet-harvest`. Proses instalasi Node.js meliputi pembaruan daftar paket, instalasi sertifikat, curl, dan gnupg, penambahan kunci GPG NodeSource, penambahan repositori NodeSource, pembaruan daftar paket lagi, dan akhirnya instalasi Node.js. Terakhir, versi Node.js yang terinstal akan ditampilkan.

In [ ]:
# Crawl Data

filename = 'etika_it.csv'
#search_keyword = 'kesehatan near:Jakarta within:15km since:2025-01-01 lang:id'
search_keyword = 'etika it since:2025-01-01 lang:id'
limit = 300

!npx -y tweet-harvest@2.6.1 -o "{filename}" -s "{search_keyword}" --tab "LATEST" -l {limit} --token {twitter_auth_token}

Sel ini digunakan untuk melakukan crawling data dari Twitter menggunakan `tweet-harvest`. Variabel `filename` menentukan nama file CSV tempat data akan disimpan. Variabel `search_keyword` berisi kata kunci pencarian tweet, dalam kasus ini adalah "etika it" sejak tanggal 1 Januari 2025 dengan bahasa Indonesia. Variabel `limit` menentukan jumlah maksimal tweet yang akan diambil. Perintah `npx -y tweet-harvest@2.6.1 ...` menjalankan proses crawling.

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path = f"tweets-data/{filename}"


# Read the CSV file into a pandas DataFrame
#df = pd.read_csv(file_path, delimiter=",")
df = pd.read_csv("tweets-data/etika_it.csv", delimiter=",")

# Display the DataFrame
display(df)

Sel ini membaca data CSV yang telah di-crawl sebelumnya ke dalam DataFrame pandas. Pertama, mengimpor pustaka `pandas`. Kemudian, menentukan path ke file CSV. Setelah itu, membaca file CSV menjadi DataFrame dan menampilkannya.

In [ ]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

Sel ini bertujuan untuk mengecek jumlah data (tweet) yang berhasil didapatkan dan disimpan dalam DataFrame. Variabel `num_tweets` akan menyimpan jumlah baris dalam DataFrame, yang kemudian akan dicetak.

## **PREPROCESSING**

Sel ini adalah judul bagian untuk tahap preprocessing data.

https://journal.umkendari.ac.id/decode/article/view/131/61
1. Case Folding
2. Tokenizing
3. Stopword Removal
4. Normalisasi
5. Stemming

Sel ini berisi daftar tahapan preprocessing yang akan dilakukan, beserta tautan ke sebuah jurnal sebagai referensi. Tahapannya meliputi: Case Folding, Tokenizing, Stopword Removal, Normalisasi, dan Stemming.

In [ ]:
#import nltk digunakan untuk mengimpor modul NLTK (Natural Language Toolkit) ke dalam program Python
#nltk.download('punkt') digunakan untuk mengunduh data yang diperlukan oleh tokenisasi Punkt dari NLTK.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

Sel ini mengimpor pustaka NLTK (Natural Language Toolkit) dan mengunduh data yang diperlukan untuk tokenisasi (`punkt` dan `punkt_tab`). NLTK adalah pustaka populer untuk pemrosesan bahasa alami.

In [ ]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
# adalah dua fungsi yang diimpor dari modul NLTK (Natural Language Toolkit) dalam Python
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

Sel ini mengimpor beberapa modul dan fungsi yang akan digunakan dalam preprocessing. `string` untuk operasi string, `re` untuk regular expression, `word_tokenize` dari NLTK untuk memecah teks menjadi kata, dan `FreqDist` dari NLTK untuk menghitung frekuensi kata.

# **CASE FOLDING**

Sel ini adalah judul untuk sub-bagian "Case Folding".

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
#untuk mengubah semua karakter dalam setiap elemen di dalam kolom (Series) menjadi huruf kecil (lowercase).
df['full_text_Case_Folding'] = df['full_text'].str.lower()


print('Case Folding Result : \n')
#print(TWEET_DATA['full_text'].head(5))
print(df['full_text_Case_Folding'].head(10))
print('\n\n\n')

Sel ini melakukan proses case folding, yaitu mengubah semua teks dalam kolom `full_text` menjadi huruf kecil. Hasilnya disimpan dalam kolom baru `full_text_Case_Folding` dan 10 baris pertama dari kolom hasil ditampilkan.

In [ ]:
# Memecah setiap baris teks menjadi list kata-kata (token) berdasarkan spasi (split()).
# x.split() hanya memisahkan berdasarkan spasi (tidak hapus tanda baca, dll)
df["full_text_Case_Folding_split"] = df["full_text_Case_Folding"].apply(lambda x: len(x.split()))
df

Sel ini menghitung jumlah kata dalam setiap baris teks di kolom `full_text_Case_Folding` setelah dipecah berdasarkan spasi. Hasilnya disimpan dalam kolom baru `full_text_Case_Folding_split`. DataFrame kemudian ditampilkan.

# **CLEANSING**

Sel ini adalah judul untuk sub-bagian "Cleansing".

**Tahapan Sebelum Preprocessing**

menghilangkan tweet special (seperti mention @username, hashtag #topik, URL, emoji, dan karakter khusus lainnya) biasanya merupakan bagian dari tahap preprocessing sebelum tokenizing, bukan bagian dari proses tokenizing itu sendiri. Namun, keduanya saling berkaitan.





Sel ini memberikan penjelasan mengenai tahapan sebelum preprocessing, yaitu menghilangkan karakter spesial seperti mention, hashtag, URL, emoji, dan karakter khusus lainnya. Proses ini penting dilakukan sebelum tokenizing.

In [ ]:
# NLTK word tokenize
# ungsi word_tokenize dalam NLTK (Natural Language Toolkit) adalah sebuah fungsi yang digunakan untuk membagi teks menjadi token-token kata individual, yang dikenal sebagai tokenisasi kata (word tokenization).
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['full_text_cleansing'] = df['full_text_Case_Folding'].apply(remove_tweet_special)

#remove number
#def remove_number(text):: Ini adalah definisi fungsi yang disebut remove_number. Fungsi ini memiliki satu parameter yaitu text, yang merupakan teks yang akan diproses.
#return re.sub(r"\d+", "", text): Pada baris ini, fungsi re.sub() digunakan untuk mengganti setiap angka dalam teks dengan string kosong (menghapus angka).
# Baris terakhir menerapkan fungsi remove_number ke setiap elemen dalam kolom
# 'full_text' dalam objek DataFrame TWEET_DATA. Fungsi apply() digunakan untuk
# menerapkan fungsi ke setiap elemen dalam kolom atau baris DataFrame.

def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_number)

def hapus_kata_underscore(teks):
    # Hapus kata yang mengandung underscore (baik awal, tengah, atau akhir)
    teks_baru = re.sub(r'\S*_\S*', '', teks)
    # Hapus spasi berlebih setelah penghapusan
    teks_baru = re.sub(r'\s+', ' ', teks_baru).strip()
    return teks_baru
df['full_text_cleansing'] = df['full_text_cleansing'].apply(hapus_kata_underscore)

#remove punctuation
#fungsi yang biasanya digunakan dalam pemrosesan teks untuk menghapus tanda baca dari suatu teks.
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_punctuation)

# remove whitespace leading & trailing
# fungsi yang digunakan untuk menghapus spasi kosong (whitespace) pada bagian awal (leading) dan akhir (trailing) dari sebuah teks.
def remove_whitespace_LT(text):
    return text.strip()

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
#fungsi yang digunakan untuk mengganti beberapa spasi kosong berturut-turut menjadi satu spasi kosong tunggal dalam sebuah teks.
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_whitespace_multiple)

# remove single char
# adalah sebuah fungsi yang digunakan untuk menghapus karakter tunggal (single character) dari sebuah teks.
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_singl_char)

display(df)

Sel ini melakukan beberapa langkah pembersihan data teks pada kolom `full_text_Case_Folding` dan hasilnya disimpan pada kolom `full_text_cleansing`. Langkah-langkahnya meliputi:
1.  `remove_tweet_special`: Menghapus tab, baris baru, backslash, karakter non-ASCII (emoji, dll.), mention, link, hashtag, dan URL yang tidak lengkap.
2.  `remove_number`: Menghapus semua angka dari teks.
3.  `hapus_kata_underscore`: Menghapus kata yang mengandung underscore.
4.  `remove_punctuation`: Menghapus semua tanda baca.
5.  `remove_whitespace_LT`: Menghapus spasi di awal dan akhir teks.
6.  `remove_whitespace_multiple`: Mengubah beberapa spasi berurutan menjadi satu spasi.
7.  `remove_singl_char`: Menghapus karakter tunggal (huruf yang berdiri sendiri).
DataFrame yang sudah dibersihkan kemudian ditampilkan.

In [ ]:
# Memecah setiap baris teks menjadi list kata-kata (token) berdasarkan spasi (split()).
# x.split() hanya memisahkan berdasarkan spasi (tidak hapus tanda baca, dll)
df["full_text_cleansing_split"] = df["full_text_cleansing"].apply(lambda x: len(x.split()))
df

Sel ini menghitung jumlah kata dalam setiap baris teks di kolom `full_text_cleansing` setelah dipecah berdasarkan spasi. Hasilnya disimpan dalam kolom baru `full_text_cleansing_split`. DataFrame kemudian ditampilkan.

# **TOKENIZING**

Sel ini adalah judul untuk sub-bagian "Tokenizing".

**Tokenizing**
adalah proses memecah teks atau string menjadi unit-unit yang lebih kecil yang disebut dengan token. Token bisa berupa kata-kata, frasa, simbol, karakter, atau unit lainnya tergantung pada tujuan dan konteks pengolahan teks.


Sel ini memberikan penjelasan mengenai proses tokenizing, yaitu memecah teks menjadi unit-unit yang lebih kecil (token) seperti kata, frasa, atau simbol.

In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['full_text_tokenizing'] = df['full_text_cleansing'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(df['full_text_tokenizing'].head())
print('\n\n\n')

Sel ini melakukan tokenisasi pada kolom `full_text_cleansing` menggunakan fungsi `word_tokenize` dari NLTK. Hasil tokenisasi (berupa list kata) disimpan dalam kolom baru `full_text_tokenizing`. Lima baris pertama dari hasil tokenisasi kemudian ditampilkan.

In [ ]:
df.to_csv("Hasil-Preprocessing(Setelah Tokenizing).csv")

Sel ini menyimpan DataFrame yang sudah melalui tahap tokenisasi ke dalam file CSV bernama "Hasil-Preprocessing(Setelah Tokenizing).csv".

Fungsi **NLTK calc frequency distribution** digunakan untuk menghitung frekuensi distribusi kata dalam sebuah teks. Frekuensi distribusi adalah distribusi statistik yang menunjukkan jumlah kemunculan tiap elemen dalam kumpulan data.

Kelas Counter adalah sebuah kelas yang menyediakan fungsionalitas untuk menghitung dan mengelola elemen-elemen yang terdapat dalam suatu iterable (seperti list, string, atau tuple)

Setelah mengimpor Counter, kita dapat membuat objek Counter yang akan menghitung frekuensi kemunculan tiap elemen dalam iterable.

Fungsi freqDist_wrapper(text) adalah fungsi buatan pengguna (user-defined function).

Di dalam fungsi itu, kamu bisa menggunakan Counter (dari collections) untuk menghitung frekuensi kata atau token dalam teks.

Jadi, from collections import Counter bisa digunakan di dalam fungsi freqDist_wrapper() untuk menghitung distribusi frekuensi.



Sel ini memberikan penjelasan mengenai fungsi `FreqDist` dari NLTK untuk menghitung frekuensi distribusi kata. Juga menjelaskan tentang kelas `Counter` dari modul `collections` yang dapat digunakan untuk tujuan serupa.

In [ ]:
from collections import Counter

# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

df['full_text_tokens_fdist'] = df['full_text_tokenizing'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(df['full_text_tokens_fdist'].head(10).apply(lambda x : x.most_common()))

Sel ini menghitung frekuensi distribusi token pada kolom `full_text_tokenizing` menggunakan `FreqDist` dari NLTK. Hasilnya disimpan dalam kolom baru `full_text_tokens_fdist`. Kemudian, 10 baris pertama dari frekuensi token yang paling umum ditampilkan.

# **STOPWORD REMOVAL**

Sel ini adalah judul untuk sub-bagian "Stopword Removal".

In [ ]:
# digunakan untuk mengimpor modul stopwords dari NLTK (Natural Language Toolkit) dalam Python. Modul stopwords menyediakan daftar kata-kata yang umumnya dianggap sebagai kata-kata "stop words" dalam pemrosesan teks.

from nltk.corpus import stopwords

Sel ini mengimpor modul `stopwords` dari NLTK, yang menyediakan daftar kata-kata umum (stop words) yang biasanya diabaikan dalam pemrosesan teks.

In [ ]:
#nltk.download('stopwords') digunakan untuk mengunduh data stop words (kata-kata yang umumnya dianggap sebagai kata-kata "stop words") dari NLTK. Stop words adalah kata-kata umum yang sering muncul dalam teks tetapi cenderung tidak memberikan informasi penting dalam pemrosesan teks, seperti kata-kata seperti "the", "is", "are", dan sebagainya.
nltk.download('stopwords')

Sel ini mengunduh data stop words dari NLTK jika belum ada. Stop words adalah kata-kata umum yang sering muncul namun kurang informatif.

In [ ]:
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
# kalo mau stopword berarti harus di tokenizing dulu, kalo tidak maka hasilnya beda
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
# adalah istilah yang digunakan untuk menambahkan kata-kata stop words tambahan ke dalam daftar stop words yang sudah ada.
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
# membaca file teks yang berisi daftar stop words menggunakan Pandas, Anda dapat menggunakan fungsi read_csv() dari Pandas dengan menggunakan pemisah (delimiter) yang sesuai.
txt_stopword = pd.read_csv("stopword.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
# Untuk mengkonversi string stop words menjadi list dan menambahkan kata-kata stop words tambahan ke dalamnya
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
# mengonversi list list_stopwords menjadi set ist_stopwords.
list_stopwords = set(list_stopwords)


#remove stopword pada list token
#Dalam script ini, diasumsikan bahwa list_stopwords sudah didefinisikan sebelumnya. Anda perlu memastikan bahwa list_stopwords berisi kata-kata stop words yang sesuai dengan kebutuhan Anda sebelum menjalankan script ini.
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['full_text_tokens_Stopword'] = df['full_text_tokenizing'].apply(stopwords_removal)


print(df['full_text_tokens_Stopword'].head())

Sel ini melakukan proses penghapusan stopword. Pertama, mengambil daftar stopword bahasa Indonesia dari NLTK. Kemudian, menambahkan stopword secara manual dan dari file `stopword.txt`. Daftar stopword tersebut diubah menjadi set untuk pencarian yang lebih efisien. Fungsi `stopwords_removal` didefinisikan untuk menghapus stopword dari list token. Fungsi ini diterapkan pada kolom `full_text_tokenizing` dan hasilnya disimpan di kolom `full_text_tokens_Stopword`. Lima baris pertama dari hasil penghapusan stopword ditampilkan.

In [ ]:
df

Sel ini menampilkan DataFrame setelah proses penghapusan stopword.

# **NORMALISASI**

Sel ini adalah judul untuk sub-bagian "Normalisasi".

Normalisasi dalam text preprocessing adalah proses mengubah teks tidak baku atau tidak konsisten menjadi bentuk yang standar/baku, agar lebih mudah dianalisis oleh model NLP.



Sel ini memberikan penjelasan mengenai normalisasi dalam preprocessing teks, yaitu proses mengubah teks tidak baku menjadi bentuk standar atau baku.

In [ ]:
# Dalam skrip ini, diasumsikan bahwa TWEET_DATA dan file Excel "normalisasi-V1.xlsx" sudah tersedia dan sesuai dengan struktur yang diharapkan.
normalizad_word = pd.read_excel("normalisasi-V1.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['tweet_normalized'] = df['full_text_tokens_Stopword'].apply(normalized_term)

df['tweet_normalized'].head(10)

Sel ini melakukan normalisasi kata. Pertama, membaca kamus normalisasi dari file Excel "normalisasi-V1.xlsx". Kamus ini berisi pasangan kata tidak baku dan bentuk bakunya. Fungsi `normalized_term` didefinisikan untuk mengganti kata tidak baku dengan bentuk bakunya berdasarkan kamus. Fungsi ini diterapkan pada kolom `full_text_tokens_Stopword` dan hasilnya disimpan di kolom `tweet_normalized`. Sepuluh baris pertama dari hasil normalisasi ditampilkan.

# **STEMMING**
Stemming adalah proses mengubah kata berimbuhan (infleksi atau turunan) menjadi bentuk dasar (stem)-nya. Tujuannya adalah menyederhanakan kata agar kata-kata dengan makna serupa dianggap sama dalam pemrosesan bahasa alami (NLP, seperti analisis sentimen, klasifikasi teks, dll).

Sel ini adalah judul untuk sub-bagian "Stemming" dan memberikan penjelasan singkat mengenai stemming, yaitu proses mengubah kata berimbuhan menjadi bentuk dasarnya.

In [ ]:
# pustaka sastrawi digunakan untuk melakukan stemming pada kata. StemmerFactory digunakan untuk membuat objek stemmer, dan kemudian metode stem() digunakan untuk melakukan stemming pada kata yang diberikan.
!pip install sastrawi

Sel ini menginstal pustaka `sastrawi` menggunakan pip. Sastrawi adalah pustaka Python untuk stemming bahasa Indonesia.

In [ ]:
# Fungsi Swifter dalam Pemrosesan Data
#Swifter adalah library Python yang dirancang untuk mempercepat pemrosesan data dengan memanfaatkan kekuatan multiprocessing.
!pip install swifter

Sel ini menginstal pustaka `swifter` menggunakan pip. Swifter digunakan untuk mempercepat penerapan fungsi pada DataFrame pandas dengan memanfaatkan multiprocessing.

In [ ]:
# import Sastrawi package
# mengimpor StemmerFactory dari Sastrawi untuk membuat objek stemmer. Kemudian, kita menggunakan swifter untuk menerapkan fungsi stemming pada kolom 'text' DataFrame menggunakan metode swifter.apply(). Hasil stemming disimpan dalam kolom baru 'stemmed_text'.
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
# menggunakan StemmerFactory dari pustaka Sastrawi untuk membuat objek stemmer. Kemudian, menggunakan metode stem() dari objek stemmer, kita melakukan stemming pada kata 'berjalan'. Hasil stemming akan dicetak, yaitu kata 'jalan'.
factory = StemmerFactory()
stemmer = factory.create_stemmer()


# stemmed
#stemmed_wrapper() adalah fungsi yang memanfaatkan objek stemmer untuk melakukan stemming pada setiap term. Kemudian, Anda melakukan iterasi pada term_dict dan menerapkan fungsi stemmed_wrapper()
# --pada setiap term untuk mendapatkan hasil stemming, yang kemudian disimpan kembali dalam term_dict. Terakhir, Anda mencetak term_dict yang berisi term-term yang telah distem.
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
# menggunakan swifter untuk mempercepat proses pemrosesan pada kolom 'tweet_normalized' dan menerapkan fungsi get_stemmed_term() pada setiap dokumen dalam kolom tersebut. Fungsi get_stemmed_term()
# --mengembalikan daftar term yang telah distem berdasarkan nilai yang ada dalam term_dict.
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tweet_tokens_stemmed'] = df['tweet_normalized'].swifter.apply(get_stemmed_term)
print(df['tweet_tokens_stemmed'])

Sel ini melakukan proses stemming. Pertama, mengimpor `StemmerFactory` dari Sastrawi dan `swifter`. Kemudian, membuat objek stemmer. Selanjutnya, membuat kamus `term_dict` yang berisi semua term unik dari kolom `tweet_normalized` dan melakukan stemming pada setiap term tersebut. Fungsi `get_stemmed_term` didefinisikan untuk mengambil bentuk dasar (stemmed) dari setiap term dalam dokumen berdasarkan `term_dict`. Fungsi ini diterapkan pada kolom `tweet_normalized` menggunakan `swifter.apply()` dan hasilnya disimpan di kolom `tweet_tokens_stemmed`. Hasil stemming kemudian ditampilkan.

In [ ]:
df.to_csv("Hasil-Akhir-Preprocessing(Setelah Stemming).csv")

Sel ini menyimpan DataFrame yang sudah melalui tahap stemming ke dalam file CSV bernama "Hasil-Akhir-Preprocessing(Setelah Stemming).csv".

In [ ]:
df.to_csv("Hasil-Akhir-Normalisasi.csv")

Sel ini menyimpan DataFrame (kemungkinan sebelum stemming, berdasarkan nama file) ke dalam file CSV bernama "Hasil-Akhir-Normalisasi.csv".

In [ ]:
# menginstal pustaka openpyxl.
# Pustaka ini merupakan salah satu pustaka populer untuk membaca dan menulis file berformat Excel (XLSX) menggunakan Python.
!pip install openpyxl

Sel ini menginstal pustaka `openpyxl` menggunakan pip. Pustaka ini digunakan untuk membaca dan menulis file Excel berformat XLSX.

In [ ]:
# menginstal pustaka xlrd.
# Pustaka ini digunakan untuk membaca file berformat Excel (XLS) menggunakan Python.
!pip install xlrd

Sel ini menginstal pustaka `xlrd` menggunakan pip. Pustaka ini digunakan untuk membaca file Excel berformat XLS (versi lama).

In [ ]:
!pip install nltk

Sel ini menginstal pustaka `nltk` menggunakan pip. Ini mungkin redundan jika NLTK sudah diinstal sebelumnya.

In [ ]:
from nltk.tokenize import sent_tokenize


Sel ini mengimpor fungsi `sent_tokenize` dari `nltk.tokenize`. Fungsi ini digunakan untuk memecah teks menjadi kalimat.

In [ ]:
df

Sel ini menampilkan DataFrame saat ini.

# **Menggabungkan Kalimat Hasil Tokenizing**

Sel ini adalah judul untuk bagian "Menggabungkan Kalimat Hasil Tokenizing".

In [ ]:
df['kalimat_tanpa_kurung'] = df['tweet_tokens_stemmed'].apply(lambda x: ' '.join(x))

Sel ini menggabungkan kembali token-token dalam kolom `tweet_tokens_stemmed` menjadi kalimat utuh, dengan setiap token dipisahkan oleh spasi. Hasilnya disimpan dalam kolom baru `kalimat_tanpa_kurung`.

In [ ]:
df

Sel ini menampilkan DataFrame setelah proses penggabungan kalimat.

In [ ]:
df.to

Sel ini sepertinya belum selesai ditulis, hanya ada `df.to` yang kemungkinan bertujuan untuk menyimpan DataFrame ke suatu format file.

**Menghitung Jumlah Kata**

Sel ini adalah judul untuk bagian "Menghitung Jumlah Kata".

In [ ]:
#Cara Ke-1
df["kalimat_tanpa_kurung_split"] = df["kalimat_tanpa_kurung"].apply(lambda x: len(x.split()))
df

#Cara Ke-2
# Hitung jumlah kata (token) dalam setiap baris (kalimat) pada kolom teks.
# word_tokenize(x) (dari nltk) lebih canggih: mengenali tanda baca dan format bahasa
# Fungsi ini tidak bisa dilakukan pada Hasil Tokenisasi (atau ada koma atau petik)
df["JumlahKata"] = df["kalimat_tanpa_kurung"].apply(lambda x: len(word_tokenize(x)))
df

Sel ini menghitung jumlah kata dalam kolom `kalimat_tanpa_kurung` dengan dua cara:
1.  Cara pertama (`kalimat_tanpa_kurung_split`): Memecah kalimat berdasarkan spasi dan menghitung jumlah hasilnya.
2.  Cara kedua (`JumlahKata`): Menggunakan `word_tokenize` dari NLTK untuk memecah kalimat menjadi token (termasuk menghitung tanda baca sebagai token terpisah jika ada) dan menghitung jumlah token tersebut.
DataFrame kemudian ditampilkan.

# **Leksikon**

Sel ini adalah judul untuk bagian "Leksikon". Bagian ini kemungkinan akan berkaitan dengan analisis sentimen berbasis leksikon.

In [ ]:
pip install vaderSentiment

Sel ini menginstal pustaka `vaderSentiment` menggunakan pip. VADER (Valence Aware Dictionary and sEntiment Reasoner) adalah model berbasis leksikon dan aturan untuk analisis sentimen.

In [ ]:
pip install numpy

Sel ini menginstal pustaka `numpy` menggunakan pip. NumPy adalah pustaka fundamental untuk komputasi numerik di Python.

In [ ]:
pip install scikit-learn

Sel ini menginstal pustaka `scikit-learn` menggunakan pip. Scikit-learn adalah pustaka populer untuk machine learning di Python.

In [ ]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import numpy as np

Sel ini mengimpor beberapa modul dan fungsi yang diperlukan untuk analisis sentimen. Mengunduh leksikon VADER dari NLTK, mengimpor `SentimentIntensityAnalyzer` dari NLTK untuk analisis sentimen VADER, `accuracy_score` dari `scikit-learn` untuk mengukur akurasi, dan `numpy`.

In [ ]:
# Create an instance of SentimentIntensityAnalyzer
model = SentimentIntensityAnalyzer()

Sel ini membuat instance dari `SentimentIntensityAnalyzer` yang akan digunakan untuk analisis sentimen VADER.

In [ ]:
import csv
import os
#find file path for lexicon bahasa indonesia
#filepath = '/sembako4.csv'

#create dictionary for positive lexicon
lexicon_positive = {}

#read the positive tsv file
with open(os.path.join('positive.tsv')) as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        lexicon_positive[word] = int(weight)

#create dictionary for negative lexicon
lexicon_negative = {}

#read the negative tsv file
with open(os.path.join('negative.tsv')) as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        lexicon_negative[word] = int(weight)

#check the dictionaries
print(lexicon_positive)
print(lexicon_negative)

Sel ini bertujuan untuk membuat leksikon sentimen positif dan negatif dari file TSV (`positive.tsv` dan `negative.tsv`). File-file tersebut dibaca, dan setiap kata beserta bobot sentimennya disimpan dalam dictionary `lexicon_positive` dan `lexicon_negative`. Kedua dictionary tersebut kemudian dicetak.

In [ ]:
df

Sel ini menampilkan DataFrame saat ini.

Untuk menambahkan tanda petik (") atau tanda kutip (') ke dalam kolom DataFrame di Python (misalnya di sekitar setiap kata atau kalimat), kamu bisa menggunakan fungsi apply() atau str.replace() atau str.join() tergantung bentuk datanya.

Berikut beberapa contoh umum:

Sel ini memberikan penjelasan mengenai cara menambahkan tanda petik atau kutip ke dalam kolom DataFrame, yang mungkin diperlukan untuk beberapa pustaka analisis sentimen.

In [ ]:
#select columns called 'Text Filtering'
df['tweet_tokens_stemmed_2'] = df['tweet_tokens_stemmed'].apply(lambda x: f'{x}')

#view new DataFrame
df

Sel ini membuat kolom baru `tweet_tokens_stemmed_2` dengan mengonversi isi kolom `tweet_tokens_stemmed` (yang berupa list) menjadi string. Ini mungkin dilakukan agar sesuai dengan format input yang diharapkan oleh VADER atau pustaka lain. DataFrame kemudian ditampilkan.

error seperti ini karena data frame tidak ada tanda kutip
cek https://github.com/agushendra7/twitter-sentiment-analysis-using-vader-and-random-forest/blob/main/labeling/vader%20sentiment.ipynb

Sel ini berisi catatan bahwa error bisa terjadi jika DataFrame tidak memiliki tanda kutip pada data teksnya, dan memberikan tautan ke repositori GitHub sebagai referensi.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

df['Score'] = df['tweet_tokens_stemmed'].apply(lambda x: vs.polarity_scores(x))
df['Compound Score'] = df['tweet_tokens_stemmed'].apply(lambda x: vs.polarity_scores(x)['compound'])

df.head()

Sel ini mencoba melakukan analisis sentimen menggunakan VADER pada kolom `tweet_tokens_stemmed` (yang berisi list token). Hasil skor polaritas disimpan di kolom `Score` dan skor compound disimpan di kolom `Compound Score`. Lima baris pertama DataFrame ditampilkan. Kemungkinan akan terjadi error karena VADER mengharapkan input string, bukan list.

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

df['Score'] = df['tweet_tokens_stemmed_2'].apply(lambda x: vs.polarity_scores(x))
df['Compound Score'] = df['tweet_tokens_stemmed_2'].apply(lambda x: vs.polarity_scores(x)['compound'])

df.head()

Sel ini melakukan analisis sentimen menggunakan VADER, namun kali ini pada kolom `tweet_tokens_stemmed_2` (yang berisi string dari list token). Hasil skor polaritas disimpan di kolom `Score` dan skor compound disimpan di kolom `Compound Score`. Lima baris pertama DataFrame ditampilkan. Ini adalah pendekatan yang lebih tepat untuk VADER.

In [ ]:
def vader_analysis(compound):
    if compound >= 0.05:
        return 'Positive'
    elif compound <=  -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['Vader Sentiment'] = df['Compound Score'].apply(vader_analysis)
df.head(10)

Sel ini mendefinisikan fungsi `vader_analysis` untuk mengklasifikasikan sentimen (Positif, Negatif, Netral) berdasarkan skor compound dari VADER. Fungsi ini diterapkan pada kolom `Compound Score` dan hasilnya disimpan di kolom `Vader Sentiment`. Sepuluh baris pertama DataFrame ditampilkan.

In [ ]:
vader_counts = df['Vader Sentiment'].value_counts()
vader_counts

Sel ini menghitung jumlah masing-masing sentimen (Positif, Negatif, Netral) dalam kolom `Vader Sentiment` dan menampilkannya.

In [ ]:
df.to_csv("bencana-alam-sentiment.csv")

Sel ini menyimpan DataFrame yang sudah berisi hasil analisis sentimen VADER ke dalam file CSV bernama "bencana-alam-sentiment.csv".

# **Disini Untuk Mengetahui Sentimen Hasil**

Sel ini adalah judul yang menandakan bagian untuk mengetahui hasil sentimen.

In [ ]:
df

Sel ini menampilkan DataFrame saat ini.

In [ ]:
df['kalimat_tanpa_kurung'] = df['tweet_tokens_stemmed'].apply(lambda x: ' '.join(x))

Sel ini menggabungkan kembali token-token dalam kolom `tweet_tokens_stemmed` menjadi kalimat utuh, dengan setiap token dipisahkan oleh spasi. Hasilnya disimpan dalam kolom `kalimat_tanpa_kurung`. Ini sepertinya duplikasi dari sel sebelumnya dengan ID `fc90ffef`.

In [ ]:
df

Sel ini menampilkan DataFrame setelah proses penggabungan kalimat (kemungkinan duplikasi).

In [ ]:
# Modul pandas digunakan untuk manipulasi dan analisis data tabular, sedangkan modul Counter digunakan untuk menghitung kemunculan elemen-elemen dalam suatu koleksi.
import pandas as pd
from collections import Counter

Sel ini mengimpor pustaka `pandas` dan `Counter` dari `collections`. Ini mungkin redundan jika sudah diimpor sebelumnya.

# **Menghitung Banyak Kata**

Sel ini adalah judul untuk bagian "Menghitung Banyak Kata".

In [ ]:
# Memisahkan kata-kata dalam teks
kata_kunci = df['kalimat_tanpa_kurung'].str.split()
kata_kunci

Sel ini memisahkan kata-kata dalam setiap kalimat di kolom `kalimat_tanpa_kurung` dan menyimpannya sebagai list kata dalam variabel `kata_kunci`. Variabel `kata_kunci` kemudian ditampilkan.

In [ ]:
df.info()

Sel ini menampilkan informasi ringkas mengenai DataFrame, termasuk tipe data setiap kolom dan jumlah nilai non-null.

# **Mengubah Nama Kolom**

Sel ini adalah judul untuk bagian "Mengubah Nama Kolom".

In [ ]:
df.rename(columns={'Vader Sentiment': 'label'}, inplace=True)
df.head()

Sel ini mengubah nama kolom `Vader Sentiment` menjadi `label` secara langsung pada DataFrame (`inplace=True`). Lima baris pertama DataFrame setelah perubahan nama kolom ditampilkan.

In [ ]:
import numpy as np
import pandas as pd
import nltk
from io import StringIO
import pytz
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ast
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import SpaceTokenizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
import os
import socket
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
#st.set_option('deprecation.showPyplotGlobalUse', False) #ini dikasih komentar biar ga error
import math
import pprint
from sklearn.svm import LinearSVC #classifier SVM
from sklearn.svm import SVC
# Menginisialisasi Sastrawi Stemmer
from wordcloud import WordCloud
factory = StemmerFactory()
stemmer = factory.create_stemmer()


Sel ini mengimpor berbagai pustaka yang umum digunakan dalam machine learning dan pemrosesan teks, termasuk `numpy`, `pandas`, `nltk`, `Sastrawi` untuk stemming, `scikit-learn` untuk berbagai fungsi machine learning (LabelEncoder, TfidfVectorizer, SVM, metrik, dll.), `matplotlib` dan `seaborn` untuk visualisasi, serta `wordcloud` untuk membuat word cloud. Objek stemmer dari Sastrawi juga diinisialisasi.

In [ ]:
"""TF-IDF"""
#Perlu dicari cara untuk membuat Label dengan TF-IDF

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['kalimat_tanpa_kurung'], df['label'], test_size=0.2, random_state=7)

df_train = pd.DataFrame()
df_train['kalimat_tanpa_kurung'] = x_train
df_train['label'] = y_train

df_test= pd.DataFrame()
df_test['kalimat_tanpa_kurung'] = x_test
df_test['label'] = y_test

df_train

df_test

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf vectorizer
tfidf=TfidfVectorizer()
tfidf.fit(df['kalimat_tanpa_kurung'].values.astype('U'))
#hitung tf-idf setiap kata pada data training
x_train_tfidf = tfidf.transform(df_train['kalimat_tanpa_kurung'].values.astype('U'))
#hitung tf-idf setiap kata pada data testing
x_test_tfidf = tfidf.transform(df_test['kalimat_tanpa_kurung'].values.astype('U'))

df_train.to_csv('tweet-train.csv', index=False)
df_train.to_csv('tweet-test.csv', index=False)

tfidf

print(x_train_tfidf)

print(x_test_tfidf)

print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

print(tfidf.vocabulary_)

Sel ini melakukan pembagian data menjadi data latih dan data uji, kemudian melakukan vektorisasi TF-IDF. 
1. Data dibagi menjadi set latih (`x_train`, `y_train`) dan set uji (`x_test`, `y_test`) dari kolom `kalimat_tanpa_kurung` (fitur) dan `label` (target).
2. DataFrame baru `df_train` dan `df_test` dibuat untuk menyimpan data latih dan uji.
3. `TfidfVectorizer` diinisialisasi dan di-fit pada seluruh data `kalimat_tanpa_kurung`.
4. Data latih dan uji ditransformasi menggunakan TF-IDF menjadi `x_train_tfidf` dan `x_test_tfidf`.
5. Data latih dan uji (sebelum TF-IDF) disimpan ke file CSV.
6. Objek `tfidf`, hasil transformasi TF-IDF, bentuknya, dan vocabulary-nya dicetak.

# **SVM**

Sel ini adalah judul untuk bagian "SVM" (Support Vector Machine).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Sel ini mengimpor pustaka yang diperlukan untuk implementasi model SVM, termasuk `pandas`, `numpy`, `seaborn`, `matplotlib`, dan modul-modul dari `scikit-learn` seperti `train_test_split`, `TfidfVectorizer`, `SVC` (Support Vector Classifier), serta metrik evaluasi (`accuracy_score`, `confusion_matrix`, `classification_report`).

In [ ]:
# Split data
x = df['kalimat_tanpa_kurung']
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Sel ini memisahkan fitur (x) dari kolom `kalimat_tanpa_kurung` dan label (y) dari kolom `label`. Kemudian, data dibagi menjadi data latih dan data uji dengan proporsi 80:20 dan `random_state=42` untuk reproduktifitas.

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

Sel ini melakukan vektorisasi TF-IDF pada data teks. `TfidfVectorizer` diinisialisasi dengan `max_features=5000` (membatasi jumlah fitur menjadi 5000 kata paling sering muncul). Vectorizer di-fit pada data latih (`x_train`) dan kemudian digunakan untuk mentransformasi data latih (`x_train_tfidf`) dan data uji (`x_test_tfidf`).

In [ ]:
SVM = SVC(kernel='linear')  # Gunakan kernel linear untuk teks
SVM.fit(x_train_tfidf, y_train)

Sel ini menginisialisasi model SVM dengan kernel linear (`SVC(kernel='linear')`) yang umumnya baik untuk data teks. Model SVM kemudian dilatih menggunakan data latih TF-IDF (`x_train_tfidf`) dan label latih (`y_train`).

In [ ]:
# Prediksi
predictions_SVM = SVM.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_SVM)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_SVM)
print("Confusion Matrix:\n", matrix)

# Heatmap
sns.heatmap(matrix, annot=True, fmt='g', cmap='Blues',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Asli')
plt.title('Confusion Matrix - SVM')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_SVM))

Sel ini melakukan prediksi menggunakan model SVM yang telah dilatih pada data uji TF-IDF (`x_test_tfidf`). Kemudian, metrik evaluasi dihitung dan ditampilkan:
- Akurasi (`accuracy_score`)
- Confusion Matrix (`confusion_matrix`), yang juga divisualisasikan sebagai heatmap menggunakan `seaborn`.
- Classification Report (`classification_report`) yang berisi precision, recall, dan f1-score untuk setiap kelas.

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions_SVM)
print("Accuracy:", accuracy)


Sel ini secara spesifik menghitung dan mencetak akurasi model SVM pada data uji.

**average	Penjelasan**

'micro'	Menghitung total TP, FP, FN dari seluruh kelas (global)
- Ingin skor global akurat, cocok untuk data seimbang

'macro'	Rata-rata metrik dari tiap kelas (semua kelas dianggap sama penting)
- Ingin adil ke semua kelas, cocok untuk data tidak seimbang

'weighted'	Rata-rata metrik tiap kelas, diberi bobot sesuai jumlah sampel tiap kelas
- Kompromi antara micro dan macro

None	Mengembalikan skor per kelas (array)
- Butuh skor per kelas secara terpisah

Sel ini memberikan penjelasan mengenai parameter `average` yang digunakan dalam perhitungan metrik seperti precision, recall, dan F1-score pada `scikit-learn`, terutama untuk masalah klasifikasi multikelas. Dijelaskan perbedaan antara `micro`, `macro`, `weighted`, dan `None`.

In [ ]:
# Calculate precision
precision = precision_score(y_test, predictions_SVM, average='macro')
print("Precision:", precision)

# Calculate recall (sensitivity)
recall = recall_score(y_test, predictions_SVM, average='macro')
print("Recall (Sensitivity):", recall)

# Calculate F1-score
f1 = f1_score(y_test, predictions_SVM, average='macro')
print("F1-Score:", f1)

Sel ini menghitung dan mencetak precision, recall, dan F1-score dengan menggunakan `average='macro'`. Penggunaan `average='macro'` berarti metrik dihitung untuk setiap kelas secara independen dan kemudian dirata-rata, memberikan bobot yang sama untuk setiap kelas.

In [ ]:
#Simpan Hasil Prediksi (Opsional)
test_prediction = pd.DataFrame()
test_prediction['text'] = x_test
test_prediction['label'] = predictions_SVM
test_prediction.to_csv('svm_sentiment_predictions.csv', index=False)

Sel ini menyimpan hasil prediksi sentimen dari model SVM pada data uji ke dalam file CSV bernama `svm_sentiment_predictions.csv`. File ini akan berisi teks asli dan label prediksi.

# **RANDOM FOREST**

Sel ini adalah judul untuk bagian "RANDOM FOREST".

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Sel ini mengimpor pustaka yang diperlukan untuk implementasi model Random Forest, termasuk `pandas`, `numpy`, `seaborn`, `matplotlib`, dan modul-modul dari `scikit-learn` seperti `train_test_split`, `TfidfVectorizer`, `RandomForestClassifier`, serta metrik evaluasi.

In [ ]:
# Pisahkan fitur dan label
x = df['kalimat_tanpa_kurung']
y = df['label']

# Split data menjadi train dan test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Sel ini memisahkan fitur (x) dari kolom `kalimat_tanpa_kurung` dan label (y) dari kolom `label`. Kemudian, data dibagi menjadi data latih dan data uji dengan proporsi 80:20 dan `random_state=42`. Ini adalah langkah yang sama seperti pada persiapan data SVM.

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

Sel ini melakukan vektorisasi TF-IDF pada data teks, sama seperti pada persiapan data SVM. `TfidfVectorizer` diinisialisasi dengan `max_features=5000`, di-fit pada data latih, dan digunakan untuk mentransformasi data latih dan uji.

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_tfidf, y_train)

Sel ini menginisialisasi model Random Forest Classifier (`RandomForestClassifier`) dengan `n_estimators=100` (jumlah pohon dalam forest) dan `random_state=42`. Model kemudian dilatih menggunakan data latih TF-IDF (`x_train_tfidf`) dan label latih (`y_train`).

In [ ]:
# Prediksi
predictions_RF = rf_model.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_RF)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_RF)
print("Confusion Matrix:\n", matrix)

# Heatmap Confusion Matrix
sns.heatmap(matrix, annot=True, fmt='g', cmap='YlGnBu',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Label Sebenarnya')
plt.title('Confusion Matrix - Random Forest')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_RF))

Sel ini melakukan prediksi menggunakan model Random Forest yang telah dilatih pada data uji TF-IDF (`x_test_tfidf`). Kemudian, metrik evaluasi dihitung dan ditampilkan:
- Akurasi (`accuracy_score`)
- Confusion Matrix (`confusion_matrix`), yang juga divisualisasikan sebagai heatmap.
- Classification Report (`classification_report`).

In [ ]:
#Simpan Hasil Prediksi (Opsional)
# Simpan prediksi ke file CSV
test_prediction = pd.DataFrame()
test_prediction['text'] = x_test
test_prediction['label'] = predictions_RF
test_prediction.to_csv('randomforest_sentiment_predictions.csv', index=False)

Sel ini menyimpan hasil prediksi sentimen dari model Random Forest pada data uji ke dalam file CSV bernama `randomforest_sentiment_predictions.csv`.

# **Naive Bayes**

Sel ini adalah judul untuk bagian "Naive Bayes".

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Sel ini mengimpor pustaka yang diperlukan untuk implementasi model Naive Bayes, termasuk `pandas`, `numpy`, `seaborn`, `matplotlib`, dan modul-modul dari `scikit-learn` seperti `train_test_split`, `TfidfVectorizer`, `MultinomialNB` (Multinomial Naive Bayes, cocok untuk data teks), serta metrik evaluasi.

In [ ]:
# Pisahkan fitur dan label
x = df['kalimat_tanpa_kurung']
y = df['label']

# Split train dan test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Sel ini memisahkan fitur (x) dari kolom `kalimat_tanpa_kurung` dan label (y) dari kolom `label`. Kemudian, data dibagi menjadi data latih dan data uji dengan proporsi 80:20 dan `random_state=42`. Langkah ini identik dengan persiapan data untuk SVM dan Random Forest.

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

Sel ini melakukan vektorisasi TF-IDF pada data teks, sama seperti pada persiapan data model sebelumnya. `TfidfVectorizer` diinisialisasi dengan `max_features=5000`, di-fit pada data latih, dan digunakan untuk mentransformasi data latih dan uji.

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(x_train_tfidf, y_train)

Sel ini menginisialisasi model Multinomial Naive Bayes (`MultinomialNB`). Model kemudian dilatih menggunakan data latih TF-IDF (`x_train_tfidf`) dan label latih (`y_train`).

In [ ]:
# Prediksi
predictions_NB = nb_model.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_NB)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_NB)
print("Confusion Matrix:\n", matrix)

# Visualisasi confusion matrix
sns.heatmap(matrix, annot=True, fmt='g', cmap='coolwarm',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Label Sebenarnya')
plt.title('Confusion Matrix - Naive Bayes')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_NB))

Sel ini melakukan prediksi menggunakan model Naive Bayes yang telah dilatih pada data uji TF-IDF (`x_test_tfidf`). Kemudian, metrik evaluasi dihitung dan ditampilkan:
- Akurasi (`accuracy_score`)
- Confusion Matrix (`confusion_matrix`), yang juga divisualisasikan sebagai heatmap.
- Classification Report (`classification_report`).

**Lain lagi**

Sel ini menandakan adanya pendekatan atau kode lain, kemungkinan variasi dari implementasi Naive Bayes atau model lain.

In [ ]:
# metode naive bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
bayes = MultinomialNB()
##tahapan untuk menjalankan training
bayes.fit(x_train_tfidf,y_train)

#menjalankan data testing
nb_result = bayes.predict(x_test_tfidf)
from sklearn import metrics
model = metrics.accuracy_score(y_test, nb_result)
print("The Accuracy is",str('{:04.2f}'.format(model*100))+'%')

from sklearn.metrics import confusion_matrix
matrix2= confusion_matrix(y_test, nb_result)
print(matrix2)

import seaborn as sns
sns.heatmap(matrix2, square= True, annot= True, cbar= False, cmap='RdBu', xticklabels=['Positif','Netral','Negatif'], yticklabels= ['Positif','Netral','Negatif'], fmt= 'g')
plt.xlabel('prediksi label')
plt.ylabel('true label')

from sklearn.metrics import classification_report
print("The classification report is:")
print(classification_report(y_test, nb_result))

Sel ini adalah implementasi lain dari model Naive Bayes.
1. Mengimpor `MultinomialNB` dan `classification_report`.
2. Menginisialisasi model `MultinomialNB` sebagai `bayes`.
3. Melatih model menggunakan data latih TF-IDF (`x_train_tfidf`) dan `y_train`.
4. Melakukan prediksi pada data uji TF-IDF (`x_test_tfidf`) dan menyimpan hasilnya di `nb_result`.
5. Menghitung dan mencetak akurasi model.
6. Menghitung dan mencetak confusion matrix, serta memvisualisasikannya dengan heatmap.
7. Mencetak classification report.

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, nb_result)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, nb_result, average='macro')
print("Precision:", precision)

# Calculate recall (sensitivity)
recall = recall_score(y_test, nb_result, average='macro')
print("Recall (Sensitivity):", recall)

# Calculate F1-score
f1 = f1_score(y_test, nb_result, average='macro')
print("F1-Score:", f1)

Sel ini menghitung dan mencetak metrik evaluasi tambahan untuk hasil prediksi Naive Bayes (`nb_result` dari sel sebelumnya), yaitu akurasi, precision (macro), recall (macro), dan F1-score (macro).

In [ ]:
# hasil

Berita = input('Masukkan Tweet : ')
result = bayes.predict(tfidf.transform([Berita]))
if(result == [1]):
  result = 'Berita yang dimasukkan merupakan berita Gempa Bumi'
elif(result==[4]):
  result = 'Berita yang dimasukkan merupakan berita Banjir'
result

Sel ini adalah contoh bagaimana model Naive Bayes yang telah dilatih (`bayes`) dapat digunakan untuk memprediksi sentimen dari input teks baru. Pengguna diminta memasukkan tweet, teks tersebut diubah menjadi representasi TF-IDF, dan model memprediksi labelnya. Ada logika `if-elif` yang tampaknya spesifik untuk kategori berita tertentu (Gempa Bumi, Banjir), yang mungkin tidak relevan untuk analisis sentimen umum (Positif, Negatif, Netral) yang dilatih sebelumnya.